In [2]:
import numpy as np

In [96]:
class FourInARow:

    def __init__(self):
        
        self.arraySize = arraySize = 42
        self.width = width = 7

        self.board = np.zeros(arraySize)
        
        self.winningCombinations = np.array([
            [0, 1, 2, 3],
            [0, width, 2 * width, 3 * width],
            [0, width + 1, 2 * width + 2, 3 * width + 3],
            [3, width + 2, 2 * width + 1, 3 * width]
        ])
        
    
    def get_action_size(self):
        return self.width

    def get_available_actions(self, board):
        actions = np.zeros(self.width)
        for i in range(self.width):
            if np.any(board[i::self.width]==0):
                actions[i] = 1
        return actions
    
    def get_board_from_player(self, player):
        return self.board * player

    def check_if_winner(self, player):
        
        arraySize = self.arraySize
        width = self.width
        
        for combo in self.winningCombinations:

            for pos in range(arraySize):
                
                #print(combo, pos)
                shifted_combo = combo + pos
                combo_rows = shifted_combo // width

                # check if i'm already outside the board
                if max(combo_rows) >= arraySize // width: continue

                # exclude combinations spanning the wrong numbers of rows
                differenceRows = max(combo_rows) - min(combo_rows)
                if differenceRows != 3 and differenceRows != 0: continue

                #console.log(pos, shifted_combo, differenceRows)

                #console.log(shifted_combo.every((el) => game[el] === player))
                if (np.array([self.board[el] for el in shifted_combo]) == player).all():
                    #print(player, " wins with ", shifted_combo)
                    return True
        
        return False
    
    def play(self, board):
        self.board = board.copy()
           
    def get_reward_for_player(self, board, player):
        arraySize = self.arraySize
        width = self.width
        
        for combo in self.winningCombinations:

            for pos in range(arraySize):
                
                shifted_combo = combo + pos
                combo_rows = shifted_combo // width

                # check if i'm already outside the board
                if max(combo_rows) >= arraySize // width: continue

                # exclude combinations spanning the wrong numbers of rows
                differenceRows = max(combo_rows) - min(combo_rows)
                if differenceRows != 3 and differenceRows != 0: continue

                if (np.array([board[el] for el in shifted_combo]) == player).all():
                    return 1.0
                elif (np.array([board[el] for el in shifted_combo]) == -1*player).all():
                    return -1.0
                
        if np.any(board==0):
            return None
        
        return 0
    
    def next_player(self):
        board = list(self.board)
        if board.count(1) == board.count(-1):
            self.player = 1
        else:
            self.player = -1
        return self.player
    
    def next_state(self, board, player, action):
        self.board = board
        for pos in range(35+action,-1,-self.width):
            if self.board[pos]==0:
                self.board[pos] = player
                break
        player = self.next_player()
        return self.board, player

coordinates

In [97]:
np.arange(42).reshape(6,7)

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39, 40, 41]])

In [98]:
fiar = FourInARow()

assert the action size

In [99]:
assert fiar.get_action_size()==7

assert the available actions

In [100]:
assert (fiar.get_available_actions(np.zeros(42))==np.ones(7)).all()
assert (fiar.get_reward_for_player(np.zeros(42),1)==None)
assert (fiar.get_reward_for_player(np.zeros(42),-1)==None)
fiar.play(np.zeros(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==False )



assert (fiar.get_available_actions(np.ones(42))==np.zeros(7)).all()
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)
fiar.play(-1*np.ones(42))
assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )

In [101]:
case = np.zeros(42)
case[-7:] = 1

columns_to_fill = [0,3]
for c in columns_to_fill:
    case[c::7] = 1


solution = np.ones(7)
solution[columns_to_fill] = 0

print(case.reshape(6,7))

[[1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [102]:
assert (fiar.get_available_actions(case)==solution).all()

fiar.play(case)

assert (fiar.check_if_winner(1)==True )
assert (fiar.check_if_winner(-1)==False )
assert (fiar.get_reward_for_player(np.ones(42),1)==1)
assert (fiar.get_reward_for_player(np.ones(42),-1)==-1)

fiar.play(case*-1)

assert (fiar.check_if_winner(1)==False )
assert (fiar.check_if_winner(-1)==True )
assert (fiar.get_reward_for_player(-np.ones(42),1)==-1)
assert (fiar.get_reward_for_player(-np.ones(42),-1)==1)

In [106]:
next_player = 1
next_state, next_player = fiar.next_state(board=np.zeros(42), player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=0)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=3)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=2)
print(next_state.reshape(6,7))

next_state, next_player = fiar.next_state(board=next_state, player=next_player, action=3)
print(next_state.reshape(6,7))

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.]]
